# NN BASICS

In [1]:
import os
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Helpers

In [170]:
def calculateOutput(layer:torch.nn.modules.linear.Linear, input_arr:np.ndarray,
            is_relu=True)->np.ndarray:
    """
    Calculates the output from a liner layer.
       N - number of samples
       O - number of outputs
       I - number of inputs

    Args:
        layer: Linear layer
        input_arr: np.Array (N X I)

    Returs
        np.array: N X O
    """
    num_input = layer.in_features
    num_output = layer.out_features
    # Get the weights
    A_tensor = list(layer.named_parameters())[0][1]
    #
    b_tensor = torch.detach((list(layer.named_parameters())[1][1])).clone()
    #
    weight_product_tensor = A_tensor.detach().matmul(input_arr)  # rows are outputs, columns are samples
    result = weight_product_tensor + b_tensor.reshape(num_output, 1)
    if is_relu:
        result = nn.ReLU()(result)
    return result.detach().numpy().T
    
######### TESTS
# Setup
test_data_arr = np.array([ [1, 2],
                      [10, 20],
                      [100, 200],
                         ], dtype=float)
input_arr = torch.tensor(test_data_arr.T).to(torch.float32)
layer = nn.Linear(2, 4)
layer_tensor = layer(input_arr.T).detach()
#
result = calculateOutput(layer, input_arr, is_relu=False)
assert(np.sum((result - layer_tensor.numpy())**2) <= 1e-5)
# With relu
result = calculateOutput(layer, input_arr, is_relu=True)
assert(np.sum((result - nn.ReLU()(layer_tensor).numpy())**2) <= 1e-5)
print("OK!")

OK!


In [156]:
dir(layer)

['T_destination',
 '__annotations__',
 '__call__',
 '__class__',
 '__constants__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_backward_hooks',
 '_backward_pre_hooks',
 '_buffers',
 '_call_impl',
 '_compiled_call_impl',
 '_forward_hooks',
 '_forward_hooks_always_called',
 '_forward_hooks_with_kwargs',
 '_forward_pre_hooks',
 '_forward_pre_hooks_with_kwargs',
 '_get_backward_hooks',
 '_get_backward_pre_hooks',
 '_get_name',
 '_is_full_backward_hook',
 '_load_from_state_dict',
 '_load_state_dict_post_hooks',
 '_load_state_dict_pre_hooks',
 '_maybe_warn_non_full_backward_hook',
 '_modules',
 '_named_members',
 '_non_per

# Get GPU

In [2]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using mps device


# Basic Network

In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(2, 3),
            nn.ReLU(),
            nn.Linear(3, 4),
            nn.ReLU(),
            nn.Linear(4, 5),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=2, out_features=3, bias=True)
    (1): ReLU()
    (2): Linear(in_features=3, out_features=4, bias=True)
    (3): ReLU()
    (4): Linear(in_features=4, out_features=5, bias=True)
  )
)


In [5]:
X = torch.rand(1, 2, 1, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([3], device='mps:0')


In [6]:
pred_probab

tensor([[0.1628, 0.1750, 0.2248, 0.2270, 0.2103]], device='mps:0',
       grad_fn=<SoftmaxBackward0>)

In [122]:
pred_probab.argmax(0)

tensor([0, 0, 0, 0, 0], device='mps:0')

In [7]:
named_parameters = list(model.named_parameters())
[(x[0], len(x[1])) for x in named_parameters]

[('linear_relu_stack.0.weight', 3),
 ('linear_relu_stack.0.bias', 3),
 ('linear_relu_stack.2.weight', 4),
 ('linear_relu_stack.2.bias', 4),
 ('linear_relu_stack.4.weight', 5),
 ('linear_relu_stack.4.bias', 5)]

# NN Operation

NN operation is a matrix multiplication. ReLU provides thresholding.
One challenge is managing the inputs to and outputs from a neuron.

# Autoencoders

In [175]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

# Set device
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

# 1. Basic Fully Connected Autoencoder
class BasicAutoencoder(nn.Module):
    def __init__(self, input_dim=784, hidden_dim=128, latent_dim=64):
        super(BasicAutoencoder, self).__init__()
        
        # Encoder
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, latent_dim),
            nn.ReLU()
        )
        
        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, input_dim),
            nn.Sigmoid()  # For image data normalized to [0,1]
        )
    
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded
    
    def encode(self, x):
        return self.encoder(x)

# 2. Convolutional Autoencoder (better for images)
class ConvAutoencoder(nn.Module):
    def __init__(self):
        super(ConvAutoencoder, self).__init__()
        
        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 16, 3, stride=2, padding=1),  # 28x28 -> 14x14
            nn.ReLU(),
            nn.Conv2d(16, 32, 3, stride=2, padding=1), # 14x14 -> 7x7
            nn.ReLU(),
            nn.Conv2d(32, 64, 7)                       # 7x7 -> 1x1
        )
        
        # Decoder
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(64, 32, 7),                      # 1x1 -> 7x7
            nn.ReLU(),
            nn.ConvTranspose2d(32, 16, 3, stride=2, padding=1), # 7x7 -> 14x14
            nn.ReLU(),
            nn.ConvTranspose2d(16, 1, 3, stride=2, padding=1, output_padding=1), # 14x14 -> 28x28
            nn.Sigmoid()
        )
    
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

# 3. Variational Autoencoder (VAE)
class VAE(nn.Module):
    def __init__(self, input_dim=784, hidden_dim=400, latent_dim=20):
        super(VAE, self).__init__()
        
        # Encoder
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc21 = nn.Linear(hidden_dim, latent_dim)  # mu
        self.fc22 = nn.Linear(hidden_dim, latent_dim)  # logvar
        
        # Decoder
        self.fc3 = nn.Linear(latent_dim, hidden_dim)
        self.fc4 = nn.Linear(hidden_dim, input_dim)
    
    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)
    
    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std
    
    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h3))
    
    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 784))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

# Loss function for VAE
def vae_loss(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD

# 4. Denoising Autoencoder
class DenoisingAutoencoder(nn.Module):
    def __init__(self, input_dim=784, hidden_dim=128, latent_dim=64):
        super(DenoisingAutoencoder, self).__init__()
        
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim, latent_dim),
            nn.ReLU()
        )
        
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim, input_dim),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

# Training function
def train_autoencoder(model, dataloader, num_epochs=10, lr=1e-3, model_type='basic'):
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    model.train()
    losses = []
    
    for epoch in range(num_epochs):
        epoch_loss = 0
        for batch_idx, (data, _) in enumerate(dataloader):
            data = data.to(device)
            
            if model_type == 'basic' or model_type == 'denoising':
                data = data.view(data.size(0), -1)  # Flatten for FC layers
                
                if model_type == 'denoising':
                    # Add noise to input
                    noise = torch.randn_like(data) * 0.3
                    noisy_data = data + noise
                    noisy_data = torch.clamp(noisy_data, 0., 1.)
                    output = model(noisy_data)
                else:
                    output = model(data)
                
                loss = criterion(output, data)
                
            elif model_type == 'conv':
                output = model(data)
                loss = criterion(output, data)
                
            elif model_type == 'vae':
                recon_batch, mu, logvar = model(data)
                loss = vae_loss(recon_batch, data, mu, logvar)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            epoch_loss += loss.item()
        
        avg_loss = epoch_loss / len(dataloader)
        losses.append(avg_loss)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')
    
    return losses

# Visualization function
def visualize_results(model, dataloader, model_type='basic', num_images=8):
    model.eval()
    with torch.no_grad():
        data, _ = next(iter(dataloader))
        data = data[:num_images].to(device)
        
        if model_type == 'basic' or model_type == 'denoising':
            data_flat = data.view(data.size(0), -1)
            if model_type == 'denoising':
                # Add noise for demonstration
                noise = torch.randn_like(data_flat) * 0.3
                noisy_data = data_flat + noise
                noisy_data = torch.clamp(noisy_data, 0., 1.)
                reconstructed = model(noisy_data)
            else:
                reconstructed = model(data_flat)
            reconstructed = reconstructed.view(-1, 1, 28, 28)
            
        elif model_type == 'conv':
            reconstructed = model(data)
            
        elif model_type == 'vae':
            reconstructed, _, _ = model(data)
            reconstructed = reconstructed.view(-1, 1, 28, 28)
    
    # Plot original and reconstructed images
    fig, axes = plt.subplots(2, num_images, figsize=(12, 4))
    for i in range(num_images):
        # Original
        axes[0, i].imshow(data[i].cpu().squeeze(), cmap='gray')
        axes[0, i].set_title('Original')
        axes[0, i].axis('off')
        
        # Reconstructed
        axes[1, i].imshow(reconstructed[i].cpu().squeeze(), cmap='gray')
        axes[1, i].set_title('Reconstructed')
        axes[1, i].axis('off')
    
    plt.tight_layout()
    plt.show()

# Example usage
if __name__ == "__main__":
    # Load MNIST dataset
    transform = transforms.Compose([
        transforms.ToTensor(),
    ])
    
    train_dataset = torchvision.datasets.MNIST(
        root='./data', train=True, download=True, transform=transform
    )
    
    train_loader = DataLoader(
        train_dataset, batch_size=128, shuffle=True
    )
    
    # Example 1: Basic Autoencoder
    print("Training Basic Autoencoder...")
    basic_ae = BasicAutoencoder().to(device)
    basic_losses = train_autoencoder(basic_ae, train_loader, num_epochs=5, model_type='basic')
    
    # Example 2: Convolutional Autoencoder
    # FIXME: Problem with the NN definition or data that is dimensioned wrong.
    #print("\nTraining Convolutional Autoencoder...")
    #conv_ae = ConvAutoencoder().to(device)
    #conv_losses = train_autoencoder(conv_ae, train_loader, num_epochs=5, model_type='conv')
    conv_losses = [np.nan]
    
    # Example 3: Variational Autoencoder
    print("\nTraining Variational Autoencoder...")
    vae = VAE().to(device)
    vae_losses = train_autoencoder(vae, train_loader, num_epochs=5, model_type='vae')
    
    # Example 4: Denoising Autoencoder
    print("\nTraining Denoising Autoencoder...")
    denoising_ae = DenoisingAutoencoder().to(device)
    denoising_losses = train_autoencoder(denoising_ae, train_loader, num_epochs=5, model_type='denoising')
    
    # Visualize results (uncomment to see plots)
    # visualize_results(basic_ae, train_loader, 'basic')
    # visualize_results(conv_ae, train_loader, 'conv')
    # visualize_results(vae, train_loader, 'vae')
    # visualize_results(denoising_ae, train_loader, 'denoising')
    
    print("\nTraining completed!")
    print(f"Final losses - Basic: {basic_losses[-1]:.4f}, Conv: {conv_losses[-1]:.4f}, "
          f"VAE: {vae_losses[-1]:.4f}, Denoising: {denoising_losses[-1]:.4f}")

Using mps device
Training Basic Autoencoder...
Epoch [1/5], Loss: 0.0500
Epoch [2/5], Loss: 0.0220
Epoch [3/5], Loss: 0.0167
Epoch [4/5], Loss: 0.0136
Epoch [5/5], Loss: 0.0118

Training Variational Autoencoder...
Epoch [1/5], Loss: 21122.5012
Epoch [2/5], Loss: 15497.6482
Epoch [3/5], Loss: 14616.2168
Epoch [4/5], Loss: 14236.7660
Epoch [5/5], Loss: 14029.5964

Training Denoising Autoencoder...
Epoch [1/5], Loss: 0.0588
Epoch [2/5], Loss: 0.0344
Epoch [3/5], Loss: 0.0300
Epoch [4/5], Loss: 0.0281
Epoch [5/5], Loss: 0.0270

Training completed!
Final losses - Basic: 0.0118, Conv: nan, VAE: 14029.5964, Denoising: 0.0270
